# Usage Instructions - Time Series Ticket Priority Forecasting

Mphasis time series ticket forecasting helps businesses assess the number of tickets that can be generate at each priority level based on historic data. This will help businesses assess the human resource required to resolve the issues and allocate more people if necessary. It uses ensemble ML algorithms with automatic model selection algorithms. This solution provides consistent and better results due to its ensemble learning approach. This solution performs automated model selection to apply the right model based on the input data.  


## Contents

1. [Prequisites](#Prerequisite)
1. [Data Dictionary](#Data-Dictionary)
1. [Set Up The Environment](#Set-up-the-environment)
1. [Create The Model](#Create-Model)
1. [Batch Transform Job](#Batch-Transform-Job)
1. [Invoke Endpoint](#Invoking-through-Endpoint)

### Prerequisites

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.


### Data Dictionary

- The input has to be a '.csv' file with 'utf-8' encoding. PLEASE NOTE: If your input .csv file is not 'utf-8' encoded, model   will not perform as expected
1. Have an unique identifier column called 'maskedsku'. eg. 'maskedsku' can be shipmentid
2. The date format of the columns should be: 'YYYY-MM-DD'

### Sample input data

In [1]:
import pandas as pd
df = pd.read_csv("sample.csv")
df.head(10)

,maskedsku,2018-08-01 12:00,2018-08-01 13:00,2018-08-01 14:00,2018-08-01 15:00,2018-08-01 16:00,2018-08-01 17:00,2018-08-01 18:00,2018-08-01 19:00,2018-08-01 20:00,...,2018-08-02 13:00,2018-08-02 14:00,2018-08-02 15:00,2018-08-02 16:00,2018-08-02 17:00,2018-08-02 18:00,2018-08-02 19:00,2018-08-02 20:00,2018-08-02 21:00,2018-08-02 22:00
0,P1,13380.82192,15244.93151,14925.20548,13585.97260,11365.47945,20060.54795,12861.36986,14945.22740,14490.37808,...,15046.35616,19864.93151,14184.98630,12370.84932,19949.58904,14228.38356,19529.55616,16279.75890,14330.95890,15056.87671
1,P2,20433.83014,10600.76712,12656.87671,20317.80822,14872.76712,15046.35616,19864.93151,14184.98630,12370.84932,...,11879.67123,12768.00000,22173.69863,16171.39726,17321.45753,15056.87671,14330.95890,16279.75890,14228.38356,19529.55616
2,P3,14866.84932,14915.34247,17090.63014,11294.46575,20696.71233,11879.67123,12768.00000,22173.69863,16171.39726,...,20060.54795,12861.36986,14945.22740,14490.37808,12208.53699,16279.75890,14228.38356,15056.87671,14330.95890,14228.38356


### Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [2]:
import sagemaker as sage
from time import gmtime, strftime
from sagemaker import get_execution_role

sess = sage.Session()
role = get_execution_role()

## Create Model

Now we use the Model Package to create a model

In [3]:
# Please use the appropriate ARN obtained after subscribing to the model to define 'model_package_arn'

model_package_arn = 'arn:aws:sagemaker:us-east-2:786796469737:model-package/mphasis-marketplace-time-series-ticketseverity-prediction'
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()
model = ModelPackage(model_package_arn=model_package_arn,
                    role = role,
                    sagemaker_session = sagemaker_session)


## Input File

Now we pull a sample input file for testing the model.

In [4]:
sample_txt="s3://aws-marketplace-mphasis-assets/ticket-severity-timeseries/sample.csv"

## Batch Transform Job

Now let's use the model built to run a batch inference job and verify it works.

In [5]:
import json 
import uuid


transformer = model.transformer(1, 'ml.m5.xlarge')
transformer.transform(sample_txt, content_type='text/csv')
transformer.wait()
#transformer.output_path
print("Batch Transform complete")


................Importing plotly failed. Interactive plots will not work.
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [22/Jun/2020 07:41:37] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [22/Jun/2020 07:41:37] "#033GET /execution-parameters HTTP/1.1#033" 404 -
  maskedsku  2018-08-01 12:00  ...  2018-08-02 21:00  2018-08-02 22:00
0        P1       13380.82192  ...       14330.95890       15056.87671
1        P2       20433.83014  ...       14228.38356       19529.55616
2        P3       14866.84932  ...       14330.95890       14228.38356

[3 rows x 36 columns]
35
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.p

2020-06-22T07:41:37.950:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % se


Batch Transform complete


## Output from Batch Transform

Note: Ensure that the following package is installed on the local system : boto3

In [6]:
import boto3
print(transformer.output_path)
bucketFolder = transformer.output_path.rsplit('/')[3]
bucket_name=transformer.output_path.rsplit('/')[2]

#print(s3bucket,s3prefix)
s3_conn = boto3.client("s3")
with open('result.csv', 'wb') as f:
    s3_conn.download_fileobj(bucket_name,bucketFolder+'/sample.csv.out', f)
    print("Output file loaded from bucket")

s3://sagemaker-us-east-2-786796469737/mphasis-marketplace-time-series-tickets-2020-06-22-07-38-44-324
Output file loaded from bucket


In [7]:
df = pd.read_csv("result.csv")
df = df.drop("Unnamed: 0", axis =1)
df.head(10)

,maskedsku,2018-08-01 12:00,2018-08-01 13:00,2018-08-01 14:00,2018-08-01 15:00,2018-08-01 16:00,2018-08-01 17:00,2018-08-01 18:00,2018-08-01 19:00,2018-08-01 20:00,...,201808041300_forecast,201808041400_forecast,201808041500_forecast,201808041600_forecast,201808041700_forecast,201808041800_forecast,201808041900_forecast,201808042000_forecast,201808042100_forecast,201808042200_forecast
0,P1,13380.82192,15244.93151,14925.20548,13585.97260,11365.47945,20060.54795,12861.36986,14945.22740,14490.37808,...,20777.300910,24557.114273,19969.675000,18783.144094,27442.921632,20993.341941,23613.696721,23485.291330,19115.233872,21151.472782
1,P2,20433.83014,10600.76712,12656.87671,20317.80822,14872.76712,15046.35616,19864.93151,14184.98630,12370.84932,...,15902.639844,16323.904487,25427.774727,19090.597960,20194.000819,18624.393636,17507.228161,19338.444427,18241.282424,20925.124813
2,P3,14866.84932,14915.34247,17090.63014,11294.46575,20696.71233,11879.67123,12768.00000,22173.69863,16171.39726,...,14870.725425,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426


## Invoking through Endpoint
This is another way of deploying the model that provides results as real time inference. Here is a sample endpoint for reference

In [8]:
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit

role = get_execution_role()

sagemaker_session = sage.Session()
bucket=sagemaker_session.default_bucket()

In [9]:
content_type='text/csv'
model_name='timeseries-ticket-severity'
real_time_inference_instance_type='ml.c4.2xlarge'

In [10]:
# Please use the appropriate ARN obtained after subscribing to the model to define 'model_package_arn'
model_package_arn = 'arn:aws:sagemaker:us-east-2:786796469737:model-package/mphasis-marketplace-time-series-ticketseverity-prediction'

In [11]:
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()

In [12]:
#Define predictor wrapper class
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type=content_type)
#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [13]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-------------!

###  1. Invoking endpoint result through CLI command

In [14]:
file_name="sample.csv"

In [15]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'text/csv' --region us-east-2 result_ticket_severity.csv

{
    "ContentType": "text/csv; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}


In [16]:
df = pd.read_csv("result_ticket_severity.csv")

df.head(10)

,Unnamed: 0,maskedsku,2018-08-01 12:00,2018-08-01 13:00,2018-08-01 14:00,2018-08-01 15:00,2018-08-01 16:00,2018-08-01 17:00,2018-08-01 18:00,2018-08-01 19:00,...,201808041300_forecast,201808041400_forecast,201808041500_forecast,201808041600_forecast,201808041700_forecast,201808041800_forecast,201808041900_forecast,201808042000_forecast,201808042100_forecast,201808042200_forecast
0,0,P1,13380.82192,15244.93151,14925.20548,13585.97260,11365.47945,20060.54795,12861.36986,14945.22740,...,20777.300910,24557.114273,19969.675000,18783.144094,27442.921632,20993.341941,23613.696721,23485.291330,19115.233872,21151.472782
1,1,P2,20433.83014,10600.76712,12656.87671,20317.80822,14872.76712,15046.35616,19864.93151,14184.98630,...,15902.639844,16323.904487,25427.774727,19090.597960,20194.000819,18624.393636,17507.228161,19338.444427,18241.282424,20925.124813
2,2,P3,14866.84932,14915.34247,17090.63014,11294.46575,20696.71233,11879.67123,12768.00000,22173.69863,...,14870.725425,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426


### 2. Invoking endpoint result through python code

In [17]:
f = open('./sample.csv', mode='r')
data=f.read()
prediction = predictor.predict(data)

In [18]:
from io import StringIO

s=str(prediction,'utf-8')
data = StringIO(s) 
df=pd.read_csv(data)
df

,Unnamed: 0,maskedsku,2018-08-01 12:00,2018-08-01 13:00,2018-08-01 14:00,2018-08-01 15:00,2018-08-01 16:00,2018-08-01 17:00,2018-08-01 18:00,2018-08-01 19:00,...,201808041300_forecast,201808041400_forecast,201808041500_forecast,201808041600_forecast,201808041700_forecast,201808041800_forecast,201808041900_forecast,201808042000_forecast,201808042100_forecast,201808042200_forecast
0,0,P1,13380.82192,15244.93151,14925.20548,13585.97260,11365.47945,20060.54795,12861.36986,14945.22740,...,20777.300910,24557.114273,19969.675000,18783.144094,27442.921632,20993.341941,23613.696721,23485.291330,19115.233872,21151.472782
1,1,P2,20433.83014,10600.76712,12656.87671,20317.80822,14872.76712,15046.35616,19864.93151,14184.98630,...,15902.639844,16323.904487,25427.774727,19090.597960,20194.000819,18624.393636,17507.228161,19338.444427,18241.282424,20925.124813
2,2,P3,14866.84932,14915.34247,17090.63014,11294.46575,20696.71233,11879.67123,12768.00000,22173.69863,...,14870.725425,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426,14870.725426
